In [13]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os

In [2]:
print("Tensorflow version: {}".format(tf.__version__))

Tensorflow version: 2.2.0


In [9]:
(train_image,train_label),(test_image,test_label)=keras.datasets.fashion_mnist.load_data()

In [ ]:
plt.imshow(train_image[0])

In [ ]:
np.max(train_image), np.max(test_image)

In [10]:
train_iamge = train_image/255
test_image = test_image/255

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])
model.fit(train_image, train_label,batch_size=128, epochs=10,validation_data=(test_image, test_label))

In [ ]:
model.evaluate(test_image, test_label,verbose=0)

In [ ]:
model.save('./model/model_save/test_model.h5')

In [ ]:
new_model = keras.models.load_model('./model/model_save/test_model.h5')

In [ ]:
new_model.summary()

In [ ]:
new_model.evaluate(test_image, test_label,verbose=0)

# 仅保存架构

In [ ]:
json_config=model.to_json()

In [ ]:
json_config

In [ ]:
reinitialized_model = tf.keras.models.model_from_json(json_config)

In [ ]:
reinitialized_model.summary()

In [ ]:
reinitialized_model.compile(optimizer='adam',
                            loss='sparse_categorical_crossentropy',
                            metrics=['acc'])

In [ ]:
reinitialized_model.evaluate(test_image, test_label,verbose=0)

# 仅保存权重

In [ ]:
weights = model.get_weights()

In [ ]:
reinitialized_model.set_weights(weights)

In [ ]:
reinitialized_model.evaluate(test_image, test_label,verbose=0)

In [ ]:
model.save_weights('./model/model_save/weights.h5')

In [ ]:
reinitialized_model.load_weights('./model/model_save/weights.h5')

In [ ]:
reinitialized_model.evaluate(test_image, test_label,verbose=0)

# 在训练期间保存检查点

In [ ]:
checkpoint_path = './model/model_save/cp.ckpt'

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                  save_weights_only=True)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

In [ ]:
model.fit(train_image, train_label, epochs=10, callbacks=[cp_callback])

In [ ]:
model.evaluate(test_image, test_label, verbose=0)

重启后运行

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [2]:
(train_image,train_label),(test_image,test_label)=keras.datasets.fashion_mnist.load_data()

In [3]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

In [5]:
model.evaluate(test_image, test_label,verbose=0)

[174.64999389648438, 0.10649999976158142]

In [6]:
model.load_weights('./model/model_save/cp.ckpt')

In [7]:
model.evaluate(test_image, test_label,verbose=0)

[0.5436093211174011, 0.8321999907493591]

# 自定义训练中保存检查点

In [3]:
model = tf.keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(10))

In [4]:
optimizer = tf.keras.optimizers.Adam()

In [5]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [6]:
def loss(model,x,y):
    y_ = model(x)
    return loss_func(y,y_)

In [8]:
train_loss = keras.metrics.Mean('train_loss',dtype=tf.float32)
train_accuracy = keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = keras.metrics.Mean('test_loss',dtype=tf.float32)
test_accuracy = keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [14]:
cp_dir = 'model/model_save/'
cp_prefix = os.path.join(cp_dir,'cpkt')

In [15]:
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 model=model
)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((train_image,train_label))
dataset = dataset.shuffle(10000).batch(32)

In [12]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels,pred)
    grads = t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels,pred)

In [16]:
def train():
    for epoch in range(5):
        for (batch, (images,labels)) in enumerate(dataset):
            train_step(model,images,labels)
        print('Epoch{} loss is {}'.format(epoch,train_loss.result()))
        print('Epoch{} Accuracy is {}'.format(epoch, train_accuracy.result()))
        train_loss.reset_states()
        train_accuracy.reset_states()
        if (epoch + 1) % 2 == 0:
            checkpoint.save(file_prefix=cp_prefix)

In [17]:
train()

Epoch0 loss is 4.063316822052002
Epoch0 Accuracy is 0.7103166580200195
Epoch1 loss is 0.6559657454490662
Epoch1 Accuracy is 0.779699981212616
Epoch2 loss is 0.5746126770973206
Epoch2 Accuracy is 0.8011500239372253
Epoch3 loss is 0.5376013517379761
Epoch3 Accuracy is 0.8106833100318909
Epoch4 loss is 0.5164250135421753
Epoch4 Accuracy is 0.8218666911125183


## 恢复模型

In [6]:
import tensorflow as tf
from tensorflow import keras
import os

In [4]:
(train_image,train_label),(test_image,test_label)=tf.keras.datasets.fashion_mnist.load_data()
train_iamge = train_image/255
test_image = test_image/255

In [7]:
model = tf.keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(10))

In [8]:
optimizer = tf.keras.optimizers.Adam()

In [9]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [10]:
def loss(model,x,y):
    y_ = model(x)
    return loss_func(y,y_)

In [11]:
train_loss = keras.metrics.Mean('train_loss',dtype=tf.float32)
train_accuracy = keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = keras.metrics.Mean('test_loss',dtype=tf.float32)
test_accuracy = keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [12]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels,pred)
    grads = t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels,pred)

In [13]:
cp_dir = 'model/model_save/'
cp_prefix = os.path.join(cp_dir,'cpkt')

In [14]:
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 model=model
)

In [15]:
tf.train.latest_checkpoint(cp_dir)

'model/model_save/cpkt-2'

In [16]:
checkpoint.restore(tf.train.latest_checkpoint(cp_dir))

In [17]:
tf.argmax(model(train_image, training=False),axis=-1).numpy()

array([9, 0, 0, ..., 3, 0, 5])

In [18]:
train_label

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [19]:
(tf.argmax(model(train_image, training=False),axis=-1).numpy()==train_label).sum()/len(train_label)

0.8195666666666667